# Install the Required libraries

In [ ]:
%pip install langchain
%pip install langchain_openai
%pip install langchain_community
%pip install pymongo
%pip install python-dotenv
%pip install azure-core
%pip install azure-cosmos
%pip install jq

In [ ]:
%pip install -U gradio
%pip install ipywidgets

# Load the environment variables 

In [1]:
# load the environment variables .env file
import os

from dotenv import load_dotenv

load_dotenv()

True

# Initialize the OpenAI client

In [2]:
import openai

openai.api_type = os.getenv("OPENAI_API_TYPE", "azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT", "https://<YOUR-OPENAI-DEPLOYMENT-NAME>.openai.azure.com/")
openai.api_version = os.getenv("OPENAI_API_VERSION", "2023-09-15-preview")
openai.api_key = os.getenv("OPENAI_API_KEY", "<YOUR-DEPLOYMENT-KEY>")

# Create Database and Collection

In [6]:
from pymongo import MongoClient

# Connection string
mongo_connection_string = os.getenv("AZURE_COSMOS_CONNECTION_STRING", "<YOUR-COSMOS-DB-CONNECTION-STRING>")
mongo_client = MongoClient(mongo_connection_string)

# Database name
db_name = os.getenv("AZURE_COSMOS_DATABASE_NAME", "ResturantChain")
db = mongo_client[db_name]

/tmp/ipykernel_78755/2767242498.py:5: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  mongo_client = MongoClient(mongo_connection_string)


In [46]:
from langchain.document_loaders.json_loader import JSONLoader

SOURCE_FILE_NAME = "./data/results.json"

loader = JSONLoader(file_path=SOURCE_FILE_NAME, jq_schema=".[]", text_content=False)
json_data = loader.load_and_split()

In [47]:
print(json_data[1])

page_content='{"category": "Smoothies", "name": "Jimmy Jam Smoothie", "description": "Berries n kale, strawberries, bananas, blueberries kale, tropical fruit blend, and dragon fruit. Our fruity tasty smoothies are blended to perfection.", "price": "5.49 USD"}' metadata={'source': '/workspaces/build-24-langchain-vcore/data/results.json', 'seq_num': 2}


In [48]:
from langchain_openai import AzureOpenAIEmbeddings

openai_embeddings_model = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME")
openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME")

azure_openai_embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    model=openai_embeddings_model,
    azure_deployment=openai_embeddings_deployment,
)

In [49]:
from langchain.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch

collection_name = os.getenv("AZURE_COSMOS_COLLECTION_NAME", "Food")
index_name = os.getenv("AZURE_COSMOS_INDEX_NAME")

collection = db[collection_name]

vector_store: AzureCosmosDBVectorSearch = AzureCosmosDBVectorSearch.from_documents(
    json_data[0:100],
    azure_openai_embeddings,
    collection=collection,
    index_name=index_name,
)

In [50]:
from langchain_community.vectorstores.azure_cosmos_db import (
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType,
)

# Read more about these variables in detail here. https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/vector-search
num_lists = 100
dimensions = 1536
similarity_algorithm = CosmosDBSimilarityType.COS
kind = CosmosDBVectorSearchType.VECTOR_IVF
m = 16
ef_construction = 64
ef_search = 40
score_threshold = 0.1

# Create the collection and the index
vector_store.create_index(
    num_lists, dimensions, similarity_algorithm, kind, m, ef_construction
)

{'raw': {'defaultShard': {'numIndexesBefore': 1,
   'numIndexesAfter': 2,
   'createdCollectionAutomatically': False,
   'ok': 1}},
 'ok': 1}

In [55]:
query = "Beef Bacon"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

In [59]:
from langchain_openai import AzureChatOpenAI

openai_chat_model = os.getenv("AZURE_OPENAI_CHAT_MODEL_NAME")
openai_chat_deployment= os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

azure_openai_chat: AzureChatOpenAI = AzureChatOpenAI(
    model=openai_chat_model,
    azure_deployment=openai_chat_deployment,
)

In [60]:
# Test the chat
chat_response = azure_openai_chat.invoke("Tell me a joke")
print(chat_response.content)

Why did the tomato turn red?

Because it saw the salad dressing!


In [61]:
order_prompt="""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [62]:
from langchain.prompts import PromptTemplate

ORDER_PROMPT_TEMPLATE = PromptTemplate.from_template(order_prompt)

In [63]:
from langchain.chains import ConversationalRetrievalChain

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5, 'score_threshold': 0.2})

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=azure_openai_chat,
    retriever=retriever,
    condense_question_prompt=ORDER_PROMPT_TEMPLATE,
    return_source_documents=False,
    verbose=False
)

In [65]:
question = "recommend me a strawberry smoothi"
chat_history = []
response = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(response['answer'])

Based on the given context, I would recommend the "Jimmy Jam Smoothie" which is a blend of strawberries, bananas, blueberries, and kale. It is priced at 5.49 USD.


In [66]:
chat_history.append((question, response['answer']))
question = "What did I just ask you about?"
response = rag_chain.invoke({"question": question, "chat_history": chat_history})

In [67]:
print(response['answer'])

There are two smoothies that include strawberries:

1. Jimmy Jam Smoothie: Berries n kale, strawberries, bananas, blueberries kale, tropical fruit blend, and dragon fruit.
2. Jameria Crush Smoothie: Detox rocks, pineapple, bananas, cranberry, kale, lemon, and berry mango blend smoothies.

Both of these smoothies include strawberries. If you like a smoothie with a detoxifying twist, then go for the Jameria Crush Smoothie. Otherwise, Jimmy Jam Smoothie is also a great choice.


In [41]:
import gradio as gr


def setup_gradio_interface(chain):    
    with gr.Blocks() as demo_interface:
        chatbot = gr.Chatbot(label="Food Ordering System")
        chat_history = gr.State([])
        lc_chat_history = gr.State([])
        msg = gr.Textbox(label="Your question")
        gr.ClearButton([msg, chatbot])
  
        def fetch_response(message, chat_history, lc_chat_history):
            response = chain.invoke({"question": message, "chat_history": lc_chat_history})
            lc_chat_history.append((message, response['answer']))
            chat_history.append([message, response["answer"]])
            return "", chat_history, lc_chat_history

        msg.submit(fetch_response, inputs=[msg, chatbot, lc_chat_history], outputs=[msg, chatbot, lc_chat_history])
    
    return demo_interface

In [42]:
food_ordering_demo = setup_gradio_interface(rag_chain)
food_ordering_demo.launch()

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
